In [1]:
import pandas as pd
from Acesso import Login
from Query import Query
from glob import glob

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

In [2]:
arquivos=glob('*.xlsx')

df=pd.read_excel(arquivos[-1])

In [3]:
df

,SKU,Produto,Qtde Caixa,B,C,D,E,F,FUNC,B2C,...,DISTRIBUIDORES,Dif.1,TABELA FUNCIONÁRIO,Dif.2,TABELA CONSUMIDOR FINAL,Dif.3,Desc C,Desc D,Desc E,Desc F
0,3,POLPINHA DE ABACAXI (100G),6,23.66,22.20,20.91,20.04,19.77,15.86,28.29,...,19.64,-1.27,14.89,-0.97,26.57,-1.72,0.0617,0.1162,0.1530,0.1644
1,12,POLPINHA DE ACEROLA (100G),6,26.84,25.18,23.72,22.74,22.42,17.99,32.10,...,21.78,-1.94,16.52,-1.47,29.47,-2.63,0.0618,0.1162,0.1528,0.1647
2,34,"IQF AMORA CONGELADOS (1,200 KG)",9,49.56,45.49,43.54,41.75,41.19,33.49,59.18,...,53.83,10.29,41.41,7.92,73.17,13.99,0.0821,0.1215,0.1576,0.1689
3,35,POLPINHA DE AMORA (100G),6,51.23,48.08,45.29,43.40,42.81,34.34,61.27,...,46.14,0.85,34.99,0.65,62.43,1.16,0.0615,0.1159,0.1528,0.1644
4,39,POLPINHA DE BANANA (100G),6,22.44,21.06,19.84,18.75,18.50,15.04,26.84,...,16.27,-3.57,12.61,-2.43,20.80,-6.04,0.0615,0.1159,0.1644,0.1756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,9630,CONGELADO POLENTA PALITO DM (1KG),10,14.13,12.90,12.54,12.36,12.19,9.78,16.79,...,11.83,-0.71,9.23,-0.55,15.85,-0.94,0.0870,0.1125,0.1253,0.1373
252,9631,CONGELADO POLENTA PALITO DM (300G),20,4.92,4.49,4.37,4.31,4.25,3.41,5.85,...,4.08,-0.29,3.19,-0.22,5.47,-0.38,0.0874,0.1118,0.1240,0.1362
253,9632,CONGELADO POLENTA TOLETE DM ( 1KG ),12,14.60,13.33,12.96,12.78,12.60,10.11,17.36,...,11.83,-1.13,9.23,-0.88,15.85,-1.51,0.0870,0.1123,0.1247,0.1370
254,9633,"AÇAI BUFALO COM MORANGO CONGELADO 1,300KG",6,19.19,17.61,17.10,16.86,16.62,12.97,22.91,...,17.54,0.44,13.30,0.33,23.50,0.59,0.0823,0.1089,0.1214,0.1339


In [20]:
#00000029,00000045,00000004,00000050

#alterar desconto

codigos=df['SKU'].unique().tolist()

conecta=sql.ConexaoSQL()

for c in codigos:
    
    try:
    
        desc=df.loc[df['SKU']==c,'Desc F'].max()
        
        querys=f"""

        UPDATE preco
        SET desc_prom={float(desc)},
        desc_max_prd={float(desc)}
        WHERE cd_prod={c} AND LTRIM(RTRIM(cd_tabela))='00000029'

        """

        sql.Salvar(querys,conecta)
        
        pass
    
    except Exception as erro:
        
        print(erro)
        
        pass
    
    #break
    
    pass


Erro: ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'nan'. (207) (SQLExecDirectW)")


In [8]:
#alteração de preço massivo
tabelas_dict={'00000029':'B','00000045':'D','00000004':'FUNC','00000050':'B2C'}

codigos=df['SKU'].unique().tolist()

conecta=sql.ConexaoSQL()

for cod,tab in tabelas_dict.items():
    
    for c in codigos:
        
        preco=df.loc[df['SKU']==c,tab].max()
        
        #Unid. VDA
        unid=df.loc[df['SKU']==c,'Unid. VDA'].max()
        
        #caixa Qtde Caixa
        caixa=df.loc[df['SKU']==c,'Qtde Caixa'].max()
        
        preco=preco if unid!='CX' else round(preco/caixa,2)

        querys=f"""

        UPDATE preco
        SET vl_preco={float(preco)}
        WHERE cd_prod={c} AND LTRIM(RTRIM(cd_tabela))='{cod}'

        """

        sql.Salvar(querys,conecta)

        #break

        pass
    
    print(f'Tabela {tab} alterada com sucesso!')
    
    pass


Tabela B alterada com sucesso!
Tabela D alterada com sucesso!
Tabela FUNC alterada com sucesso!
Tabela B2C alterada com sucesso!
